In [24]:
import requests
import json

def generate_question(chunk, model_name="llama3.2"):
    """
    Generate a single question-answer pair from a given text chunk using an Ollama model.
    
    Args:
        chunk (str): The text chunk to generate a QA pair from
        model_name (str): The name of the Ollama model to use (default: "llama3.2")
        
    Returns:
        str: Generated question-answer pair in tuple format
    """
    # Ollama API endpoint (default is localhost on port 11434)
    api_url = "http://localhost:11434/api/generate"
    
    # Create the prompt for question and answer generation
    prompt = f"""
    You are an expert question-answer pair generator. Your task is to create exactly ONE high-quality question and its corresponding answer based on the provided text chunk.

    TEXT CHUNK:
    {chunk}

    INSTRUCTIONS:
    1. Generate ONE question-answer pair that captures key information from the text chunk.
    2. The answer must be comprehensive, accurate, and fully supported by the text.
    3. The answer should be 1 sentence long - informative but concise.
    4. The question should belong to one of these categories:
       - Basic factual questions (e.g., "When was Carnegie Mellon University founded?")
       - Document-enhanced questions that benefit from specific textual context (e.g., "What is the name of the annual pickle festival held in Pittsburgh?")
       - Specific information questions that require exact details from the text (e.g., "When was the Pittsburgh Soul Food Festival established?")
       - Time-sensitive questions related to events, schedules, or dates (e.g., "Who is performing at City Theatre on May 15th?")
    5. Vary the question types across different chunks to ensure diversity.
    
    OUTPUT FORMAT:
    Return ONLY a Python tuple with proper formatting: ("Question text", "Answer text")
    - Include proper quotes and parentheses
    - No explanations, comments, or additional text
    - No numbering or prefixes
    
    Generate a question-answer pair now:
    """
    
    # Prepare the request payload
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        # Make the API request to Ollama
        response = requests.post(api_url, json=payload)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        # Parse the response
        result = response.json()
        return result["response"].strip()
    
    except requests.exceptions.RequestException as e:
        print(f"Error communicating with Ollama: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error processing Ollama response: {e}")
        return None

In [25]:
chunk = "The Los Angeles Dodgers won the World Series in 2020. The games were played in Arlington, Texas, at Globe Life Field due to the COVID-19 pandemic."

questions = generate_question(chunk, model_name="gemma3:12b")
print(questions)

("Where did the Los Angeles Dodgers play the World Series in 2020?", "The Los Angeles Dodgers played the World Series in 2020 in Arlington, Texas, at Globe Life Field due to the COVID-19 pandemic.")


In [14]:
import re
tuple_match = re.search(r'\(\".*?\", \".*?\"\)', questions)
tuple_str = tuple_match.group(0)
tuple_str
eval(tuple_str)

('What year did the Los Angeles Dodgers win the World Series?',
 'The Los Angeles Dodgers won the World Series in 2020.')

In [68]:
# questions.strip().split("\n")

In [21]:
## read csv file
import pandas as pd
from tqdm import tqdm
df = pd.read_csv("all_combined.csv", lineterminator='\n')
print(len(df["text"]))



12370


In [30]:
def generate_questions_for_dataset(df, model_name="llama3.3:latest", max_attempts=5, save_path="all_combined.csv"):
    """
    Generate questions for each text in the dataframe using the specified model.
    
    Args:
        df: DataFrame containing a 'text' column
        model_name: Name of the model to use for question generation
        max_attempts: Maximum number of attempts for each text
        save_path: Path to save the updated dataframe
        
    Returns:
        Updated DataFrame with a new column containing generated questions
    """

    
    # Create a list to store all questions
    all_questions = []
    all_answers = []
    # Create a file to log generation results
    log_file = open(f"{model_name}_question_generation_log.txt", "w")

    # Add tqdm progress bar
    for i in tqdm(range(len(df)), desc=f"Generating questions with {model_name}"):
        # if i % 100 == 0:
        #     print("current row index: ", i)
        attempts = 0
        question = None
        
        while attempts < max_attempts:
            try:
                response = generate_question(df["text"][i], model_name=model_name)
                # print("The text is: ", df["text"][i])
                if model_name == "deepseek-r1":
                    tuple_match = re.search(r'\(\".*?\", \".*?\"\)', response)
                    tuple_str = tuple_match.group(0)
                    question, answer = eval(tuple_str)
                else:
                    question, answer = eval(response)
                if question and answer:  # If we got a valid response
                    # print("-------- ---------")
                    # print(question)
                    # print(answer)
                    # print("-------- ---------")
                    break
                raise Exception("Empty response received")
                
            except Exception as e:
                attempts += 1
                print(f"Attempt {attempts} failed: {str(e)}")
                if attempts == max_attempts:
                    print(f"Failed to generate question after {max_attempts} attempts")
        
        # Add the question to our list (empty string if generation failed)
        if question and answer:
            all_questions.append(question)
            all_answers.append(answer)
            log_file.write(f"{i}\t{question}\t{answer}\n")
        else:
            all_questions.append("")
            all_answers.append("")
            log_file.write(f"{i}\t\n")
    # Close the log file
    log_file.close()

    # After collecting all questions, add them as a new column
    column_name = f"{model_name}_question"
    df[column_name] = all_questions
    column_name = f"{model_name}_answer"
    df[column_name] = all_answers

    # Save the final dataframe once at the end
    df.to_csv(save_path, index=False)
    
    return df

In [31]:
generate_questions_for_dataset(df, model_name="gemma3:12b", save_path="all_combined_gemma3.csv")

Generating questions with gemma3:12b:   0%|          | 57/12370 [00:50<2:41:35,  1.27it/s]

Attempt 1 failed: unterminated string literal (detected at line 1) (<string>, line 1)


Generating questions with gemma3:12b:   0%|          | 58/12370 [00:52<3:34:46,  1.05s/it]

Attempt 1 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
Attempt 2 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
Attempt 3 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
Attempt 4 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)


Generating questions with gemma3:12b:   0%|          | 59/12370 [00:58<8:23:53,  2.46s/it]

...

Attempt 1 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
Attempt 2 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
Attempt 3 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
Attempt 4 failed: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)


Generating questions with gemma3:12b: 100%|█████████▉| 12328/12370 [2:42:40<00:50,  1.21s/it]

Error communicating with Ollama: 500 Server Error: Internal Server Error for url: http://localhost:11434/api/generate
Attempt 1 failed: eval() arg 1 must be a string, bytes or code object


Generating questions with gemma3:12b: 100%|██████████| 12370/12370 [2:43:25<00:00,  1.26it/s]


,source,text,llama3.2_question_text,llama3.2_answer,gemma3:12b_question,gemma3:12b_answer
0,https://www.heinzhistorycenter.org/event/irish...,Skip Main NavigationSkip to site alert\nHeinz ...,What is the date of the Irish Genealogy Workshop?,"Tuesday, Mar. 4, 2025",What dates will the one-on-one consultation se...,"The individual, one-on-one consultation sessio..."
1,https://www.heinzhistorycenter.org/event/one-o...,Skip Main NavigationSkip to site alert\nHeinz ...,What is a personal consultation with the Ulste...,A 30-minute consultation session where you wil...,What is the cost of a single 30-minute consult...,Each 30-minute consultation session is priced ...
2,https://www.heinzhistorycenter.org/event/histo...,Skip Main NavigationSkip to site alert\nHeinz ...,When does History Uncorked: Mirror Ball take p...,"Friday, Mar. 7, 2025",What is the date and time of the History Uncor...,History Uncorked: Mirror Ball will be held on ...
3,https://www.heinzhistorycenter.org/event/speak...,Skip Main NavigationSkip to site alert\nHeinz ...,What is the title of Jason A. Cherry,Factor of Ambition,When will Jason A. Cherry's lecture on 'Willia...,Jason A. Cherry’s lecture will take place on S...
4,https://www.heinzhistorycenter.org/event/gut-y...,Skip Main NavigationSkip to site alert\nHeinz ...,What does Gut Yontif mean?,Have a good holiday!,What does the Yiddish greeting 'Gut Yontif' me...,"According to the text, 'Gut Yontif' is a Yiddi..."
...,...,...,...,...,...,...
12365,2024_Operating_Budget.pdf,Anticipated 2028 Issuance ($49 million plus 1%...,('What is the City of Pittsburgh Operating Bud...,NaN,What is the amount of debt service for the Cit...,The City of Pittsburgh Operating Budget for Fi...
12366,2024_Operating_Budget.pdf,Pension Funding 318,"(""What is Pension Funding 318?"", ""A report on ...",NaN,What is the title of the text chunk provided?,The text chunk is titled 'Pension Funding 318'.
12367,2024_Operating_Budget.pdf,City of Pittsburgh Operating Budget Fiscal Ye...,"(""What is the Pension Funding for City of Pitt...",NaN,What is the subject of document 319 within the...,Document 319 within the City of Pittsburgh Ope...
12368,2024_Operating_Budget.pdf,City of Pittsburgh Operating Budget Fiscal Ye...,"(""Pension Funding in City of Pittsburgh Operat...",NaN,What is the subject of the City of Pittsburgh'...,The City of Pittsburgh Operating Budget for Fi...


In [34]:
# generate_questions_for_dataset(df, model_name="deepseek-r1", save_path="all_combined_R1.csv")

In [32]:
df = pd.read_csv("all_combined_gemma3.csv", lineterminator='\n')
df.head()

,source,text,llama3.2_question_text,llama3.2_answer,gemma3:12b_question,gemma3:12b_answer
0,https://www.heinzhistorycenter.org/event/irish...,Skip Main NavigationSkip to site alert\nHeinz ...,What is the date of the Irish Genealogy Workshop?,"Tuesday, Mar. 4, 2025",What dates will the one-on-one consultation se...,"The individual, one-on-one consultation sessio..."
1,https://www.heinzhistorycenter.org/event/one-o...,Skip Main NavigationSkip to site alert\nHeinz ...,What is a personal consultation with the Ulste...,A 30-minute consultation session where you wil...,What is the cost of a single 30-minute consult...,Each 30-minute consultation session is priced ...
2,https://www.heinzhistorycenter.org/event/histo...,Skip Main NavigationSkip to site alert\nHeinz ...,When does History Uncorked: Mirror Ball take p...,"Friday, Mar. 7, 2025",What is the date and time of the History Uncor...,History Uncorked: Mirror Ball will be held on ...
3,https://www.heinzhistorycenter.org/event/speak...,Skip Main NavigationSkip to site alert\nHeinz ...,What is the title of Jason A. Cherry,Factor of Ambition,When will Jason A. Cherry's lecture on 'Willia...,Jason A. Cherry’s lecture will take place on S...
4,https://www.heinzhistorycenter.org/event/gut-y...,Skip Main NavigationSkip to site alert\nHeinz ...,What does Gut Yontif mean?,Have a good holiday!,What does the Yiddish greeting 'Gut Yontif' me...,"According to the text, 'Gut Yontif' is a Yiddi..."


In [33]:
# Filter out rows with NaN values in gemma3:12b_question or gemma3:12b_answer columns
df_filtered = df.dropna(subset=['gemma3:12b_question', 'gemma3:12b_answer'])

# Display the filtered dataframe
print(f"Original dataframe shape: {df.shape}")
print(f"Filtered dataframe shape: {df_filtered.shape}")
df_filtered.head()


Original dataframe shape: (12370, 6)
Filtered dataframe shape: (12352, 6)


,source,text,llama3.2_question_text,llama3.2_answer,gemma3:12b_question,gemma3:12b_answer
0,https://www.heinzhistorycenter.org/event/irish...,Skip Main NavigationSkip to site alert\nHeinz ...,What is the date of the Irish Genealogy Workshop?,"Tuesday, Mar. 4, 2025",What dates will the one-on-one consultation se...,"The individual, one-on-one consultation sessio..."
1,https://www.heinzhistorycenter.org/event/one-o...,Skip Main NavigationSkip to site alert\nHeinz ...,What is a personal consultation with the Ulste...,A 30-minute consultation session where you wil...,What is the cost of a single 30-minute consult...,Each 30-minute consultation session is priced ...
2,https://www.heinzhistorycenter.org/event/histo...,Skip Main NavigationSkip to site alert\nHeinz ...,When does History Uncorked: Mirror Ball take p...,"Friday, Mar. 7, 2025",What is the date and time of the History Uncor...,History Uncorked: Mirror Ball will be held on ...
3,https://www.heinzhistorycenter.org/event/speak...,Skip Main NavigationSkip to site alert\nHeinz ...,What is the title of Jason A. Cherry,Factor of Ambition,When will Jason A. Cherry's lecture on 'Willia...,Jason A. Cherry’s lecture will take place on S...
4,https://www.heinzhistorycenter.org/event/gut-y...,Skip Main NavigationSkip to site alert\nHeinz ...,What does Gut Yontif mean?,Have a good holiday!,What does the Yiddish greeting 'Gut Yontif' me...,"According to the text, 'Gut Yontif' is a Yiddi..."
